In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [2]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = [load_array[0][20], load_array[0][50]]
inh = [load_array[1][20], load_array[1][50]]

folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

file_ = 'up.pickle'

c_var = [0,1]
if len(folder) == 4:
    if folder[3] == 'e':
        p_var = [0]
    elif folder[3] == 'i':
        p_var = [1]
else:
    p_var = [0,1]
    
print(p_var)

step1 = 1
step2 = 10
step3 = 100

/1_i
[1]


In [3]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [4]:
d_array = np.arange(20., 300, 0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i] 

In [5]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-12
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
else:
    factor_ws = 0.
    factor_we = 1.
    
maxC = [1000., -1000., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [6]:
bestControl_ = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_ = np.array( [[None] * len(t_pen)] * len(exc) )
cost_ = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_ = np.array( [[None] * len(t_pen)] * len(exc) )
weights_ = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_ = np.array( [[None] * len(t_pen)] * len(exc) )

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

print(bestControl_.shape)

(2, 2800)


In [7]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]

file found


In [7]:
# get initial parameters and target states
# -------  20 0.4500000000000001 0.4750000000000002
#-------  50 0.47500000000000014 0.6000000000000003

i_range = [0]
wp = 1./20.
data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]

-------  0 0.4500000000000001 0.4750000000000002


In [9]:
# get uncontrolled cost

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(wp, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = cost_init_[0]

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  0.05 0.0 0.0
interpolate adjoint :  True True True
initial cost =  2326.5451767678856
RUN  0 , total integrated cost =  2326.5451767678856
Gradient descend method:  None
RUN  0 , total integrated cost =  2326.5451767678856
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [10]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.

    cgv = None
    j = 0
    max_it = 1000

    while not convergence_[i][j]:

        dur = d_array[0]
        aln.params.duration = dur
        cost.setParams(wp, 0., 1.)
        weights_[i][j] = [wp, 0., 1.]
        
        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_[i][j]) == type(None) or np.amax(np.abs(bestControl_[i][j])) < 1e-6:
            control0 = aln.getZeroControl()
        else:
            control0 = bestControl_[i][j][:,:,n_pre-1:-n_post+1]

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if max_it > 1:
            if cost_[i][j][2] == 0.:
                convergence_[i][j] = True
                continue

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  0.05 0.0 1.0
interpolate adjoint :  True True True
initial cost =  2326.5451767678856
RUN  0 , total integrated cost =  2326.5451767678856
Gradient descend method:  None
RUN  1 , total integrated cost =  141.03501879983116
RUN  2 , total integrated cost =  74.80687081135231
RUN  3 , total integrated cost =  61.23239840300713
RUN  4 , total integrated cost =  56.0116382121431
RUN  5 , total integrated cost =  52.991297270009824
RUN  6 , total integrated cost =  50.89936029563149
RUN  7 , total integrated cost =  49.38304823846002
RUN  8 , total integrated cost =  48.22909446708677
RUN  9 , total integrated cost =  47.279160990378145
RUN  10 , total integrated cost =  46.516356675762964
RUN  11 , total integrated cost =  45.89361999946502
RUN  12 , total integrated cost =  45.38765042017683
RUN  13 , total integrated cost =  44.947654433676924
RUN  14 , total integrated cost =  44.58559300951404
RUN  15 , total integrated

In [12]:
tol = 1e-8

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1

    while j < 2001:
        
        if convergence_[i][j]:
            j += step
            continue

        print('-----', j)

        dur = round(d_array[j],1)
        aln.params.duration = dur
        max_it = int(1000)  

        setinit(initVars[i], aln)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_[i][j]) == type(None):
            control0 = aln.getZeroControl()
            control0[:,:,:-step] = bestControl_[i][j-step][:,:,n_pre-1:-n_post+1]

        else:
            control0 = bestControl_[i][j][:,:,n_pre-1:-n_post+1]
            
        weights_[i][j] = cost.getParams()

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_[i][j][-1] == 0.:
            convergence_[i][j] = True
            if j >= 10:
                step = step2
            if j >=100:
                step = step3

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

-------  0 0.4500000000000001 0.4750000000000002
----- 2
interpolate adjoint :  True True True
initial cost =  14.435171201988677
RUN  0 , total integrated cost =  14.435171201988677
Gradient descend method:  None
RUN  1 , total integrated cost =  14.10989865263063
RUN  2 , total integrated cost =  14.001027163597197
RUN  3 , total integrated cost =  13.961446490443148
RUN  4 , total integrated cost =  13.930009962731194
RUN  5 , total integrated cost =  13.897873854502699
RUN  6 , total integrated cost =  13.870199615286449
RUN  7 , total integrated cost =  13.845002309401263
RUN  8 , total integrated cost =  13.821859445183675
RUN  9 , total integrated cost =  13.800518962378195
RUN  10 , total integrated cost =  13.780173373685445
RUN  11 , total integrated cost =  13.761055057787955
RUN  12 , total integrated cost =  13.744741103995615
RUN  13 , total integrated cost =  13.729061913588515
RUN  14 , total integrated cost =  13.714130873159592
RUN  15 , total integrated cost =  13.69

RUN  5 , total integrated cost =  11.401810468967405
RUN  6 , total integrated cost =  11.383304174418456
RUN  7 , total integrated cost =  11.366458046051195
RUN  8 , total integrated cost =  11.352412383136139
RUN  9 , total integrated cost =  11.340028991218903
RUN  10 , total integrated cost =  11.328455030310657
RUN  11 , total integrated cost =  11.319936019189479
RUN  12 , total integrated cost =  11.312795001138431
RUN  13 , total integrated cost =  11.306534474077049
RUN  14 , total integrated cost =  11.300542141012524
RUN  15 , total integrated cost =  11.29450899626932
RUN  16 , total integrated cost =  11.287927511106627
RUN  17 , total integrated cost =  11.281554913138386
RUN  18 , total integrated cost =  11.2759575352475
RUN  19 , total integrated cost =  11.271386637390393
RUN  20 , total integrated cost =  11.266903176859966
RUN  30 , total integrated cost =  11.219920783140456
RUN  40 , total integrated cost =  11.188815943008885
RUN  50 , total integrated cost =  1

RUN  3 , total integrated cost =  10.793586723217727
RUN  4 , total integrated cost =  10.793586723217121
RUN  5 , total integrated cost =  10.79358672321651
RUN  6 , total integrated cost =  10.79358672321591
RUN  7 , total integrated cost =  10.793586723215315
RUN  8 , total integrated cost =  10.793586723214725
RUN  9 , total integrated cost =  10.793586723214135
RUN  10 , total integrated cost =  10.79358672321356
RUN  11 , total integrated cost =  10.793586723212982
RUN  12 , total integrated cost =  10.793586723212394
RUN  13 , total integrated cost =  10.79358672321182
RUN  14 , total integrated cost =  10.793586723211245
RUN  15 , total integrated cost =  10.793586723210638
RUN  16 , total integrated cost =  10.793586723210016
RUN  17 , total integrated cost =  10.793586723209366
RUN  18 , total integrated cost =  10.793586723208755
RUN  19 , total integrated cost =  10.793586723208122
RUN  20 , total integrated cost =  10.79358672320751
RUN  30 , total integrated cost =  10.79

RUN  110 , total integrated cost =  10.544710968005322
RUN  120 , total integrated cost =  10.54281790687139
RUN  130 , total integrated cost =  10.53913961086252
RUN  140 , total integrated cost =  10.538052333435601
RUN  150 , total integrated cost =  10.537050422525873
RUN  160 , total integrated cost =  10.536062748181266
RUN  170 , total integrated cost =  10.535397155275467
RUN  180 , total integrated cost =  10.5348611767185
RUN  190 , total integrated cost =  10.534522769410618
RUN  200 , total integrated cost =  10.534219761760074
RUN  300 , total integrated cost =  10.529303275177748
RUN  400 , total integrated cost =  10.528971249854969
RUN  500 , total integrated cost =  10.528889174466316
RUN  600 , total integrated cost =  10.528863140430978
RUN  700 , total integrated cost =  10.52885388996291
RUN  800 , total integrated cost =  10.528850083469191
RUN  900 , total integrated cost =  10.52884870856741
RUN  1000 , total integrated cost =  10.528848121429448
RUN  1000 , tot

RUN  110 , total integrated cost =  10.499264735463193
RUN  120 , total integrated cost =  10.498514160803623
RUN  130 , total integrated cost =  10.497877048243616
RUN  140 , total integrated cost =  10.497325373786223
RUN  150 , total integrated cost =  10.49644228689654
RUN  160 , total integrated cost =  10.494026271355553
RUN  170 , total integrated cost =  10.493487383128226
RUN  180 , total integrated cost =  10.493110689697776
RUN  190 , total integrated cost =  10.492746709377363
RUN  200 , total integrated cost =  10.492532437888814
RUN  300 , total integrated cost =  10.491698503411266
RUN  400 , total integrated cost =  10.49154111133828
RUN  500 , total integrated cost =  10.491505470805274
RUN  600 , total integrated cost =  10.49149910302742
RUN  700 , total integrated cost =  10.491498655043022
Control only changes marginally.
RUN  712 , total integrated cost =  10.491498645952841
Improved over  712  iterations in  30.10377105997759  seconds by  4.844648406096226  perce

...

RUN  2 , total integrated cost =  10.346528489582225
RUN  3 , total integrated cost =  10.346528489580875
RUN  4 , total integrated cost =  10.346528489579613
RUN  5 , total integrated cost =  10.346528489578079
RUN  6 , total integrated cost =  10.346528489576643
RUN  7 , total integrated cost =  10.346528489575311
RUN  8 , total integrated cost =  10.346528489574068
RUN  9 , total integrated cost =  10.34652848957256
RUN  10 , total integrated cost =  10.346528489571137
RUN  11 , total integrated cost =  10.346528489569833
RUN  12 , total integrated cost =  10.346528489568609
RUN  13 , total integrated cost =  10.346528489567131
RUN  14 , total integrated cost =  10.346528489565724
RUN  15 , total integrated cost =  10.346528489564442
RUN  16 , total integrated cost =  10.346528489563235
RUN  17 , total integrated cost =  10.346528489561784
RUN  18 , total integrated cost =  10.346528489560399
RUN  19 , total integrated cost =  10.346528489559136
RUN  20 , total integrated cost =  10

RUN  2 , total integrated cost =  10.346528512835937
RUN  3 , total integrated cost =  10.346528512696674
RUN  4 , total integrated cost =  10.346528512561083
RUN  5 , total integrated cost =  10.346528512423765
RUN  6 , total integrated cost =  10.346528512289574
RUN  7 , total integrated cost =  10.34652851215418
RUN  8 , total integrated cost =  10.346528512021372
RUN  9 , total integrated cost =  10.346528511887888
RUN  10 , total integrated cost =  10.34652851175644
RUN  11 , total integrated cost =  10.346528511624827
RUN  12 , total integrated cost =  10.346528511494727
RUN  13 , total integrated cost =  10.346528511364975
RUN  14 , total integrated cost =  10.3465285112362
RUN  15 , total integrated cost =  10.346528511108282
RUN  16 , total integrated cost =  10.346528510980814
RUN  17 , total integrated cost =  10.346528510854702
RUN  18 , total integrated cost =  10.346528510728536
RUN  19 , total integrated cost =  10.346528510604207
RUN  20 , total integrated cost =  10.34

RUN  1 , total integrated cost =  10.346586531004208
RUN  2 , total integrated cost =  10.346586024139041
RUN  3 , total integrated cost =  10.346585511721726
RUN  4 , total integrated cost =  10.346585012289394
RUN  5 , total integrated cost =  10.346584509605327
RUN  6 , total integrated cost =  10.346584017407487
RUN  7 , total integrated cost =  10.346583524099016
RUN  8 , total integrated cost =  10.346583039017021
RUN  9 , total integrated cost =  10.346582555039408
RUN  10 , total integrated cost =  10.346582076977004
RUN  11 , total integrated cost =  10.34658160227216
RUN  12 , total integrated cost =  10.3465811314525
RUN  13 , total integrated cost =  10.34658066620794
RUN  14 , total integrated cost =  10.346580202324207
RUN  15 , total integrated cost =  10.346579746140081
RUN  16 , total integrated cost =  10.346579289069236
RUN  17 , total integrated cost =  10.34657884178545
RUN  18 , total integrated cost =  10.346578391372109
RUN  19 , total integrated cost =  10.3465

RUN  2 , total integrated cost =  19.33386666427792
RUN  3 , total integrated cost =  14.138365188755802
RUN  4 , total integrated cost =  13.204607682697782
RUN  5 , total integrated cost =  12.85652866339197
RUN  6 , total integrated cost =  12.701962684061854
RUN  7 , total integrated cost =  12.614238804542623
RUN  8 , total integrated cost =  12.552141056118483
RUN  9 , total integrated cost =  12.50479133051691
RUN  10 , total integrated cost =  12.469500717317903
RUN  11 , total integrated cost =  12.440348597559334
RUN  12 , total integrated cost =  12.414628106684336
RUN  13 , total integrated cost =  12.391070054962507
RUN  14 , total integrated cost =  12.368998126786888
RUN  15 , total integrated cost =  12.34810820349024
RUN  16 , total integrated cost =  12.328273267545478
RUN  17 , total integrated cost =  12.30917461377903
RUN  18 , total integrated cost =  12.290618829231079
RUN  19 , total integrated cost =  12.272529081696884
RUN  20 , total integrated cost =  12.254

RUN  2 , total integrated cost =  10.346528489757114
RUN  3 , total integrated cost =  10.346528489754311
RUN  4 , total integrated cost =  10.346528489751758
RUN  5 , total integrated cost =  10.346528489750039
RUN  6 , total integrated cost =  10.346528489748467
RUN  7 , total integrated cost =  10.346528489745694
RUN  8 , total integrated cost =  10.346528489743203
RUN  9 , total integrated cost =  10.3465284897415
RUN  10 , total integrated cost =  10.346528489739962
RUN  11 , total integrated cost =  10.346528489737219
RUN  12 , total integrated cost =  10.34652848973478
RUN  13 , total integrated cost =  10.346528489733103
RUN  14 , total integrated cost =  10.346528489731325
RUN  15 , total integrated cost =  10.346528489729081
RUN  16 , total integrated cost =  10.346528489726792
RUN  17 , total integrated cost =  10.346528489725074
RUN  18 , total integrated cost =  10.346528489723385
RUN  19 , total integrated cost =  10.346528489721095
RUN  20 , total integrated cost =  10.3

In [12]:
bestControl_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
cost_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
weights_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_maxW = np.array( [[None] * len(t_pen)] * len(exc) )

file_maxW = '1_up_Wmax.pickle'

In [13]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0

    while not convergence_maxW[i][j]:

        dur = d_array[0]
        aln.params.duration = dur        
        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        if type(bestControl_maxW[i][j]) == type(None):
            max_it = 10
            control0 = bestControl_[i][0][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] - sum(costnode_[i][j][0][0][:]) ) / sum( costnode_[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = int(100)

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 2272.7839105786
interpolate adjoint :  True True True
initial cost =  46530.90353535787
RUN  0 , total integrated cost =  46530.90353535787
Gradient descend method:  None
RUN  1 , total integrated cost =  34031.451398442914
RUN  2 , total integrated cost =  26548.802710260043
RUN  3 , total integrated cost =  22349.251994034454
RUN  4 , total integrated cost =  21483.98977104431
RUN  5 , total integrated cost =  21389.740243430926
RUN  6 , total integrated cost =  21364.70222356082
RUN  7 , total integrated cost =  21358.669906745836
RUN  8 , total integrated cost =  21357.221276240874
RUN  9 , total integrated cost =  21357.034569218922
RUN  10 , total integrated cost =  21356.920097201015
RUN  10 , total integrated cost =  21356.920097201015
Improved over  10  iterations in  0.5877161137759686  seconds by  54.10164326387444  percent.
cost splitting :  [  0.         463.35999513] [8.66025667e-13 2.02692140e+01]

RUN  8 , total integrated cost =  46528.71418825559
RUN  9 , total integrated cost =  46528.70999164329
RUN  10 , total integrated cost =  46528.70729232521
RUN  11 , total integrated cost =  46528.705410043905
RUN  12 , total integrated cost =  46528.70399446872
RUN  13 , total integrated cost =  46528.70303471488
RUN  14 , total integrated cost =  46528.70220185259
RUN  15 , total integrated cost =  46528.701728604276
RUN  16 , total integrated cost =  46528.70122804697
RUN  17 , total integrated cost =  46528.701001624155
RUN  18 , total integrated cost =  46528.70068997913
RUN  19 , total integrated cost =  46528.700624662764
RUN  20 , total integrated cost =  46528.70062146364
Control only changes marginally.
RUN  26 , total integrated cost =  46528.70061391968
Improved over  26  iterations in  1.4414783790707588  seconds by  0.004734319067139836  percent.
cost splitting :  [  0.         463.35999513] [8.66025667e-13 2.02692140e+01]
set cost params:  1.0 0.0 8464.8641823704
interp

RUN  15 , total integrated cost =  43496.71213742536
RUN  16 , total integrated cost =  43496.711436542086
RUN  17 , total integrated cost =  43496.71103325709
RUN  18 , total integrated cost =  43496.71071219212
RUN  19 , total integrated cost =  43496.710451441315
RUN  20 , total integrated cost =  43496.710250861084
RUN  30 , total integrated cost =  43496.709800116805
Control only changes marginally.
RUN  33 , total integrated cost =  43496.70980008986
Improved over  33  iterations in  1.3478362979367375  seconds by  0.16140666385629743  percent.
cost splitting :  [  0.         433.55724877] [ 0.         13.56960261]
set cost params:  1.0 0.0 8844.277095192772
interpolate adjoint :  True True True
initial cost =  43567.029889575904
RUN  0 , total integrated cost =  43567.029889575904
Gradient descend method:  None
RUN  1 , total integrated cost =  43549.132398025315
RUN  2 , total integrated cost =  43549.1123982789
RUN  3 , total integrated cost =  43549.09925621533
RUN  4 , total

RUN  80 , total integrated cost =  43566.994018996105
RUN  90 , total integrated cost =  43566.991429654525
RUN  100 , total integrated cost =  43566.98915951842
RUN  100 , total integrated cost =  43566.98915951842
Improved over  100  iterations in  4.419979986734688  seconds by  9.348825841470898e-05  percent.
cost splitting :  [  0.         433.55724877] [ 0.         13.56960261]
set cost params:  1.0 0.0 9015.81457317547
interpolate adjoint :  True True True
initial cost =  43567.029889575904
RUN  0 , total integrated cost =  43567.029889575904
Gradient descend method:  None
RUN  1 , total integrated cost =  43567.02956478099
RUN  2 , total integrated cost =  43567.02923370771
RUN  3 , total integrated cost =  43567.02890830891
RUN  4 , total integrated cost =  43567.028591004484
RUN  5 , total integrated cost =  43567.02827743291
RUN  6 , total integrated cost =  43567.02796064381
RUN  7 , total integrated cost =  43567.02764360671
RUN  8 , total integrated cost =  43567.027341107

RUN  4 , total integrated cost =  43567.029588707446
RUN  5 , total integrated cost =  43567.02951997624
RUN  6 , total integrated cost =  43567.029442807456
RUN  7 , total integrated cost =  43567.02936938071
RUN  8 , total integrated cost =  43567.02929996545
RUN  9 , total integrated cost =  43567.0292287464
RUN  10 , total integrated cost =  43567.02915960836
RUN  11 , total integrated cost =  43567.02908965932
RUN  12 , total integrated cost =  43567.02902164235
RUN  13 , total integrated cost =  43567.02895210312
RUN  14 , total integrated cost =  43567.028886025684
RUN  15 , total integrated cost =  43567.02882387308
RUN  16 , total integrated cost =  43567.02875155291
RUN  17 , total integrated cost =  43567.02868137096
RUN  18 , total integrated cost =  43567.02861975195
RUN  19 , total integrated cost =  43567.02855583882
RUN  20 , total integrated cost =  43567.02849061591
RUN  30 , total integrated cost =  43567.02790300351
RUN  40 , total integrated cost =  43567.027406215

...

RUN  14 , total integrated cost =  43567.029889572346
RUN  15 , total integrated cost =  43567.029889572186
RUN  16 , total integrated cost =  43567.029889566256
RUN  17 , total integrated cost =  43567.02988956065
RUN  18 , total integrated cost =  43567.02988956025
RUN  19 , total integrated cost =  43567.02988955999
RUN  20 , total integrated cost =  43567.02988955973
RUN  30 , total integrated cost =  43567.02988953223
RUN  40 , total integrated cost =  43567.029889513215
RUN  50 , total integrated cost =  43567.02988950817
RUN  60 , total integrated cost =  43567.02988949937
RUN  70 , total integrated cost =  43567.02988948296
RUN  80 , total integrated cost =  43567.02988947228
RUN  90 , total integrated cost =  43567.02988946048
RUN  100 , total integrated cost =  43567.02988945534
RUN  100 , total integrated cost =  43567.02988945534
Improved over  100  iterations in  4.03578283265233  seconds by  2.767279738691286e-10  percent.
cost splitting :  [  0.         433.55724877] [ 0

RUN  70 , total integrated cost =  43567.029889524616
RUN  80 , total integrated cost =  43567.02988952165
RUN  90 , total integrated cost =  43567.02988951891
RUN  100 , total integrated cost =  43567.02988951049
RUN  100 , total integrated cost =  43567.02988951049
Improved over  100  iterations in  3.9460740303620696  seconds by  1.5013768006610917e-10  percent.
cost splitting :  [  0.         433.55724877] [ 0.         13.56960261]
set cost params:  1.0 0.0 9023.755646612513
interpolate adjoint :  True True True
initial cost =  43567.029889575904
RUN  0 , total integrated cost =  43567.029889575904
Gradient descend method:  None
RUN  1 , total integrated cost =  43567.029889575766
RUN  2 , total integrated cost =  43567.029889575664
RUN  3 , total integrated cost =  43567.029889575555
RUN  4 , total integrated cost =  43567.02988957546
RUN  5 , total integrated cost =  43567.02988957535
RUN  6 , total integrated cost =  43567.02988957521
RUN  7 , total integrated cost =  43567.0298

RUN  9 , total integrated cost =  43567.02988955215
RUN  10 , total integrated cost =  43567.02988954804
RUN  11 , total integrated cost =  43567.02988954657
RUN  12 , total integrated cost =  43567.02988954557
RUN  13 , total integrated cost =  43567.0298895393
RUN  14 , total integrated cost =  43567.02988953368
RUN  15 , total integrated cost =  43567.02988953155
RUN  16 , total integrated cost =  43567.0298895294
RUN  17 , total integrated cost =  43567.02988952188
RUN  18 , total integrated cost =  43567.02988951452
RUN  19 , total integrated cost =  43567.029889503785
RUN  20 , total integrated cost =  43567.0298894934
RUN  30 , total integrated cost =  43567.0298894684
RUN  40 , total integrated cost =  43567.029889467514
RUN  50 , total integrated cost =  43567.02988946498
RUN  60 , total integrated cost =  43567.02988945008
RUN  70 , total integrated cost =  43567.02988942259
RUN  80 , total integrated cost =  43567.02988942002
RUN  90 , total integrated cost =  43567.02988936

RUN  4 , total integrated cost =  43567.029889575635
RUN  5 , total integrated cost =  43567.02988957558
RUN  6 , total integrated cost =  43567.02988957552
RUN  7 , total integrated cost =  43567.029889575446
RUN  8 , total integrated cost =  43567.02988957536
RUN  9 , total integrated cost =  43567.02988957531
RUN  10 , total integrated cost =  43567.029889575206
RUN  11 , total integrated cost =  43567.029889575126
RUN  12 , total integrated cost =  43567.02988957506
RUN  13 , total integrated cost =  43567.029889574966
RUN  14 , total integrated cost =  43567.029889574886
RUN  15 , total integrated cost =  43567.02988957475
RUN  16 , total integrated cost =  43567.02988957209
RUN  17 , total integrated cost =  43567.02988956924
RUN  18 , total integrated cost =  43567.02988956917
RUN  19 , total integrated cost =  43567.029889569036
RUN  20 , total integrated cost =  43567.029889569
RUN  30 , total integrated cost =  43567.02988956473
RUN  40 , total integrated cost =  43567.029889

RUN  15 , total integrated cost =  43567.029889567726
RUN  16 , total integrated cost =  43567.02988956758
RUN  17 , total integrated cost =  43567.02988956739
RUN  18 , total integrated cost =  43567.02988956733
RUN  19 , total integrated cost =  43567.02988956727
RUN  20 , total integrated cost =  43567.02988956705
RUN  30 , total integrated cost =  43567.02988956338
RUN  40 , total integrated cost =  43567.02988956266
RUN  50 , total integrated cost =  43567.0298895619
RUN  60 , total integrated cost =  43567.029889560676
RUN  70 , total integrated cost =  43567.02988955936
RUN  80 , total integrated cost =  43567.02988955824
RUN  90 , total integrated cost =  43567.02988955603
RUN  100 , total integrated cost =  43567.02988955508
RUN  100 , total integrated cost =  43567.02988955508
Improved over  100  iterations in  3.9818216171115637  seconds by  4.779110440722434e-11  percent.
cost splitting :  [  0.         433.55724877] [ 0.         13.56960261]
set cost params:  1.0 0.0 9023.

In [14]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 1
    step = step1

    while j < 2001:
        
        if j >= 1000:
            step = step2
        
        if convergence_maxW[i][j]:
            j += step
            continue

        print('-----', j)

        dur = round(d_array[j],1)
        aln.params.duration = dur
        max_it = int(10000)  

        setinit(initVars[i], aln)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_maxW[i][j]) == type(None):
            control0 = aln.getZeroControl()
            control0[:,:,:-step] = bestControl_maxW[i][j-step][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                          - sum(costnode_maxW[i][j-step][0][0][:]) ) / sum( costnode_maxW[i][j-step][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()

        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                           - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_maxW[i][j][0][0][:2], costnode_maxW[i][j][2][0][:2])

    with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)          

-------  0 0.4500000000000001 0.4750000000000002
----- 1
set cost params:  1.0 0.0 10089.714386880672
interpolate adjoint :  True True True
initial cost =  46753.069056426255
RUN  0 , total integrated cost =  46753.069056426255
Gradient descend method:  None
RUN  1 , total integrated cost =  46577.0775374452
RUN  2 , total integrated cost =  46550.26129696144
RUN  3 , total integrated cost =  46530.56213423701
RUN  4 , total integrated cost =  46529.87539571795
RUN  5 , total integrated cost =  46529.84432886582
RUN  6 , total integrated cost =  46529.820580213025
RUN  7 , total integrated cost =  46529.80314065338
RUN  8 , total integrated cost =  46529.789201029525
RUN  9 , total integrated cost =  46529.77954073057
RUN  10 , total integrated cost =  46529.771432604575
RUN  11 , total integrated cost =  46529.766249692475
RUN  12 , total integrated cost =  46529.76185292995
RUN  13 , total integrated cost =  46529.75912595655
RUN  14 , total integrated cost =  46529.75693264374
RUN  

RUN  1 , total integrated cost =  53012.901066613136
Control only changes marginally.
RUN  1 , total integrated cost =  53012.901066613136
Improved over  1  iterations in  0.13403187785297632  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 15
set cost params:  1.0 0.0 4217605663849394.0
interpolate adjoint :  True True True
initial cost =  53475.900890274206
RUN  0 , total integrated cost =  53475.900890274206
Gradient descend method:  None
RUN  1 , total integrated cost =  53475.900890274206
Control only changes marginally.
RUN  1 , total integrated cost =  53475.900890274206
Improved over  1  iterations in  0.13503493554890156  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 16
set cost params:  1.0 0.0 4481288008618835.5
interpolate adjoint :  True True True
initial cost =  53938.90071393531
RUN  0 , total integrated cost =  53938.90071393531
Gradient 

RUN  1 , total integrated cost =  61346.89789251274
Control only changes marginally.
RUN  1 , total integrated cost =  61346.89789251274
Improved over  1  iterations in  0.13413435965776443  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 33
set cost params:  1.0 0.0 8963887869699242.0
interpolate adjoint :  True True True
initial cost =  61809.89771617383
RUN  0 , total integrated cost =  61809.89771617383
Gradient descend method:  None
RUN  1 , total integrated cost =  61809.89771617383
Control only changes marginally.
RUN  1 , total integrated cost =  61809.89771617383
Improved over  1  iterations in  0.1337738512083888  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 34
set cost params:  1.0 0.0 9227570214468676.0
interpolate adjoint :  True True True
initial cost =  62272.897539834914
RUN  0 , total integrated cost =  62272.897539834914
Gradient desce

RUN  1 , total integrated cost =  69680.89471841237
Control only changes marginally.
RUN  1 , total integrated cost =  69680.89471841237
Improved over  1  iterations in  0.1419149124994874  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 51
set cost params:  1.0 0.0 1.3710170075549082e+16
interpolate adjoint :  True True True
initial cost =  70143.89454207342
RUN  0 , total integrated cost =  70143.89454207342
Gradient descend method:  None
RUN  1 , total integrated cost =  70143.89454207342
Control only changes marginally.
RUN  1 , total integrated cost =  70143.89454207342
Improved over  1  iterations in  0.13655712641775608  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 52
set cost params:  1.0 0.0 1.3973852420318532e+16
interpolate adjoint :  True True True
initial cost =  70606.89436573454
RUN  0 , total integrated cost =  70606.89436573454
Gradient

RUN  1 , total integrated cost =  78014.89154431196
Control only changes marginally.
RUN  1 , total integrated cost =  78014.89154431196
Improved over  1  iterations in  0.14234429504722357  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 69
set cost params:  1.0 0.0 1.8456452281398932e+16
interpolate adjoint :  True True True
initial cost =  78477.89136797306
RUN  0 , total integrated cost =  78477.89136797306
Gradient descend method:  None
RUN  1 , total integrated cost =  78477.89136797306
Control only changes marginally.
RUN  1 , total integrated cost =  78477.89136797306
Improved over  1  iterations in  0.13817837927490473  seconds by  0.0  percent.
cost splitting :  [    0.         46299.40650284] [8.66025667e-13 1.63030984e-14]
----- 70
set cost params:  1.0 0.0 1.8720134626168364e+16
interpolate adjoint :  True True True
initial cost =  78940.89119163415
RUN  0 , total integrated cost =  78940.89119163415
Gradien

...

RUN  1 , total integrated cost =  615797.8232610746
Control only changes marginally.
RUN  1 , total integrated cost =  615797.8232610746
Improved over  1  iterations in  0.23004470486193895  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1330
set cost params:  1.0 0.0 1.191661473389033e+18
interpolate adjoint :  True True True
initial cost =  620132.9050290405
RUN  0 , total integrated cost =  620132.9050290405
Gradient descend method:  None
RUN  1 , total integrated cost =  620132.9050290405
Control only changes marginally.
RUN  1 , total integrated cost =  620132.9050290405
Improved over  1  iterations in  0.23079941608011723  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1340
set cost params:  1.0 0.0 1.200614635881135e+18
interpolate adjoint :  True True True
initial cost =  624467.9867970066
RUN  0 , total integrated cost =  624467.9867970066
Gradi

RUN  1 , total integrated cost =  685159.1315485293
Control only changes marginally.
RUN  1 , total integrated cost =  685159.1315485293
Improved over  1  iterations in  0.24297929741442204  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1490
set cost params:  1.0 0.0 1.33491207326266e+18
interpolate adjoint :  True True True
initial cost =  689494.2133164951
RUN  0 , total integrated cost =  689494.2133164951
Gradient descend method:  None
RUN  1 , total integrated cost =  689494.2133164951
Control only changes marginally.
RUN  1 , total integrated cost =  689494.2133164951
Improved over  1  iterations in  0.24398772604763508  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1500
set cost params:  1.0 0.0 1.343865235754762e+18
interpolate adjoint :  True True True
initial cost =  693829.2950844611
RUN  0 , total integrated cost =  693829.2950844611
Gradie

RUN  1 , total integrated cost =  754520.4398359836
Control only changes marginally.
RUN  1 , total integrated cost =  754520.4398359836
Improved over  1  iterations in  0.2556201135739684  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1650
set cost params:  1.0 0.0 1.4781626731362854e+18
interpolate adjoint :  True True True
initial cost =  758855.5216039491
RUN  0 , total integrated cost =  758855.5216039491
Gradient descend method:  None
RUN  1 , total integrated cost =  758855.5216039491
Control only changes marginally.
RUN  1 , total integrated cost =  758855.5216039491
Improved over  1  iterations in  0.25678445491939783  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1660
set cost params:  1.0 0.0 1.487115835628387e+18
interpolate adjoint :  True True True
initial cost =  763190.6033719152
RUN  0 , total integrated cost =  763190.6033719152
Gradi

RUN  1 , total integrated cost =  828216.8298914037
Control only changes marginally.
RUN  1 , total integrated cost =  828216.8298914037
Improved over  1  iterations in  0.2635888308286667  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1820
set cost params:  1.0 0.0 1.6303664355020137e+18
interpolate adjoint :  True True True
initial cost =  832551.9116593696
RUN  0 , total integrated cost =  832551.9116593696
Gradient descend method:  None
RUN  1 , total integrated cost =  832551.9116593696
Control only changes marginally.
RUN  1 , total integrated cost =  832551.9116593696
Improved over  1  iterations in  0.2679137997329235  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1830
set cost params:  1.0 0.0 1.6393195979941153e+18
interpolate adjoint :  True True True
initial cost =  836886.9934273355
RUN  0 , total integrated cost =  836886.9934273355
Gradi

RUN  1 , total integrated cost =  897578.1381788581
Control only changes marginally.
RUN  1 , total integrated cost =  897578.1381788581
Improved over  1  iterations in  0.2746407128870487  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1980
set cost params:  1.0 0.0 1.773617035375641e+18
interpolate adjoint :  True True True
initial cost =  901913.2199468242
RUN  0 , total integrated cost =  901913.2199468242
Gradient descend method:  None
RUN  1 , total integrated cost =  901913.2199468242
Control only changes marginally.
RUN  1 , total integrated cost =  901913.2199468242
Improved over  1  iterations in  0.27451742999255657  seconds by  0.0  percent.
cost splitting :  [    0.         43350.27849709] [0.00000000e+00 2.43305256e-13]
----- 1990
set cost params:  1.0 0.0 1.782570197867743e+18
interpolate adjoint :  True True True
initial cost =  906248.3017147902
RUN  0 , total integrated cost =  906248.3017147902
Gradie